# load file

In [1]:
base_dir = "/home/datamind/can/BASIL/"

In [2]:
import sys
import os
import math
os.chdir(base_dir)
sys.path.append(base_dir)

In [3]:
import torch
from basil.select_device import device
torch.device(device)

device(type='cuda', index=0)

In [4]:
from basil.preprocessing.prep_pytorch import PytorchPrepWordLevel
from basil.preprocessing.prep_pytorch_syllable import PytorchPrepWordLevelWithSyllable
from basil.evaluation.noise import NoiseGenerator
from sklearn.metrics import f1_score
import torch
import pickle
from basil.select_device import device
from allennlp.modules.elmo import Elmo, batch_to_ids


# BEST2010

In [5]:
import pickle
import pandas as pd
from basil.preprocessing.prep_best_corpus import Best2010NECTEC
from basil.preprocessing.prep_pytorch import PytorchPrepWordLevel

best2010  = Best2010NECTEC()
prep_torch = PytorchPrepWordLevel(best2010.train_data, best2010.val_data, best2010.test_data)
y_true_pos = [prep_torch.pos_to_index[word] for text in prep_torch.pos_test for word in text]
y_true_ner = [prep_torch.ner_to_index[word] for text in prep_torch.ner_test for word in text]
stress_best = pickle.load( open( base_dir+"stress_best2010.p", "rb" ) )
error10 = [item for sublist in stress_best.error10   for item in sublist] 
error20 = [item for sublist in stress_best.error20   for item in sublist] 
error30 = [item for sublist in stress_best.error30   for item in sublist] 
error40 = [item for sublist in stress_best.error40   for item in sublist] 
error50 = [item for sublist in stress_best.error50   for item in sublist] 
error60 = [item for sublist in stress_best.error60   for item in sublist] 
error70 = [item for sublist in stress_best.error70   for item in sublist] 
error80 = [item for sublist in stress_best.error80   for item in sublist] 
error90 = [item for sublist in stress_best.error90   for item in sublist] 
error100 = [item for sublist in stress_best.error100   for item in sublist] 

/home/datamind/can/BASIL/basil/preprocessing/prep_best_corpus.py:9: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(ymlfile)


Preprocessing BEST2010 corpus
Done!


In [20]:
from basil.models.multitasks.wordlvl_lstm import LSTMTagger
from basil.models.multitasks.elmo_lstm_pretrained import LSTMTagger as ELMO
from basil.models.multitasks.BiLSTM_B import LSTMTagger as BiLSTM_B
from basil.models.multitasks.BILSTM_UBC import LSTMTagger as BiLSTM_UBC
from basil.models.multitasks.BiLSTM_BA import LSTMTagger as BiLSTM_BA
from basil.models.multitasks.BiLSTM_BCA import LSTMTagger as BiLSTM_BCA
from basil.models.multitasks.BiLSTM_BCAD import LSTMTagger as BiLSTM_BCAD
CHAR_EMBEDDING_DIM = 100
WORD_EMBEDDING_DIM = 128
HIDDEN_DIM = 128

model = LSTMTagger(WORD_EMBEDDING_DIM,HIDDEN_DIM,\
                           len(prep_torch.char_to_index),len(prep_torch.word_to_index),\
                           len(prep_torch.pos_to_index),len(prep_torch.ner_to_index)).to(device)
model.load_state_dict(torch.load("weights/model_bilstm_baseline_seed1.p"))
prep_torch = PytorchPrepWordLevel(best2010.train_data, best2010.val_data, best2010.test_data)

In [7]:
text = ['วัน','นี้','ฉัน','รู้สึก','เมื่อย','มาก']
cases = [['วัน','นี้','ฉัน','รู้สึก','เมือย','มาก'],
         ['วัน','นี้','ฉัน','รูสึก','เมือย','มาก'],
        ['วั','นี','ฉั','รู้สึก','เมือย','มา'],
        ['วั','นี','ฉั','รูสึก','เมือย','มา']]
w_in = prep_torch.prepare_sequence_test_word(text)
pred_pos, pred_ner = model(w_in)

In [28]:
ans = []
for pos in pred_pos:
    pos=pos.data.tolist()
    pos = pos.index(max(pos))
    pos = prep_torch.index_to_pos[pos]
    ans.append(pos)

* วัน - NN - common nouns
* นี้ - DDEM - Demonstrative Determiner -
* ฉัน - PPER (Personal Pronoun)
* รู้สึก - VV - main verbs in clauses verb form
* เมื่อย - VV - serial verbs
* มาก - ADV - adverbs modifying Verb/Adj/other adverbs

In [29]:
ans

['NN', 'NR', 'PU', 'NR', 'VV', 'VV']

In [23]:
model_base = LSTMTagger(WORD_EMBEDDING_DIM,HIDDEN_DIM,\
                           len(prep_torch.char_to_index),len(prep_torch.word_to_index),\
                           len(prep_torch.pos_to_index),len(prep_torch.ner_to_index)).to(device)
model_base.load_state_dict(torch.load("weights/model_bilstm_baseline_seed1.p"))

model_elmo_pretrained_small_seed1 = ELMO(CHAR_EMBEDDING_DIM,WORD_EMBEDDING_DIM,HIDDEN_DIM,\
                           len(prep_torch.char_to_index),len(prep_torch.word_to_index),\
                           len(prep_torch.pos_to_index),len(prep_torch.ner_to_index)).to(device)
model_elmo_pretrained_small_seed1.load_state_dict(torch.load("weights/model_elmo_pretrained_small_seed1.p"))

model_bilstm_UNK_seed1 = LSTMTagger(WORD_EMBEDDING_DIM,HIDDEN_DIM,\
                           len(prep_torch.char_to_index),len(prep_torch.word_to_index),\
                           len(prep_torch.pos_to_index),len(prep_torch.ner_to_index)).to(device)
model_bilstm_UNK_seed1.load_state_dict(torch.load("weights/model_bilstm_UNK_seed1.p"))


model_bilstm_backoff_unk_seed1 = BiLSTM_B(CHAR_EMBEDDING_DIM,WORD_EMBEDDING_DIM,HIDDEN_DIM,\
                           len(prep_torch.char_to_index),len(prep_torch.word_to_index),\
                           len(prep_torch.pos_to_index),len(prep_torch.ner_to_index)).to(device)
model_bilstm_backoff_unk_seed1.load_state_dict(torch.load("weights/model_bilstm_backoff_unk_seed1.p"))

model_bilstm_BA_seed1 = BiLSTM_BA(CHAR_EMBEDDING_DIM,WORD_EMBEDDING_DIM,HIDDEN_DIM,\
                           len(prep_torch.char_to_index),len(prep_torch.word_to_index),\
                           len(prep_torch.pos_to_index),len(prep_torch.ner_to_index)).to(device)
model_bilstm_BA_seed1.load_state_dict(torch.load("weights/model_bilstm_BA_seed1.p"))

model_bilstm_UBC_seed1 =  BiLSTM_UBC(CHAR_EMBEDDING_DIM,WORD_EMBEDDING_DIM,HIDDEN_DIM,\
                           len(prep_torch.char_to_index),len(prep_torch.word_to_index),prep_torch.prefix_size,\
                           len(prep_torch.pos_to_index),len(prep_torch.ner_to_index)).to(device)
model_bilstm_UBC_seed1.load_state_dict(torch.load("weights/model_bilstm_UBC_seed1.p"))


model_bilstm_BCAD_clp_seed1 = BiLSTM_BCAD(CHAR_EMBEDDING_DIM,WORD_EMBEDDING_DIM,HIDDEN_DIM,\
                           len(prep_torch.char_to_index),len(prep_torch.word_to_index),prep_torch.prefix_size,\
                           len(prep_torch.pos_to_index),len(prep_torch.ner_to_index)).to(device)
model_bilstm_BCAD_clp_seed1.load_state_dict(torch.load("weights/model_bilstm_BCAD_clp_seed1.p"))




In [43]:
print("BiLSTM-BaseLine")
for case in cases:
    w_in = prep_torch.prepare_sequence_test_word(case)   
    pred_pos, pred_ner = model_base(w_in)
    ans = []
    for pos in pred_pos:
        pos=pos.data.tolist()
        pos = pos.index(max(pos))
        pos = prep_torch.index_to_pos[pos]
        ans.append(pos)
    print(case)    
    print(" & ".join(ans))

BiLSTM-BaseLine
['วัน', 'นี้', 'ฉัน', 'รู้สึก', 'เมือย', 'มาก']
NN & DDEM & PPER & VV & VA & ADV
['วัน', 'นี้', 'ฉัน', 'รูสึก', 'เมือย', 'มาก']
NN & DDEM & P & VV & ADV & ADV
['วั', 'นี', 'ฉั', 'รู้สึก', 'เมือย', 'มา']
NR & NN & NR & VV & VV & VV
['วั', 'นี', 'ฉั', 'รูสึก', 'เมือย', 'มา']
FWN & NN & NR & NR & VV & VV


* วัน - NN - common nouns
* นี้ - DDEM - Demonstrative Determiner -
* ฉัน - PPER (Personal Pronoun)
* รู้สึก - VV - main verbs in clauses verb form
* เมื่อย - VV - serial verbs
* มาก - ADV - adverbs modifying Verb/Adj/other adverbs

In [39]:
print("BiLSTM-UNK")
for case in cases:
    w_in = prep_torch.prepare_sequence_test_word(case)   
    pred_pos, pred_ner = model_bilstm_UNK_seed1(w_in)
    ans = []
    for pos in pred_pos:
        pos=pos.data.tolist()
        pos = pos.index(max(pos))
        pos = prep_torch.index_to_pos[pos]
        ans.append(pos)
    print(case)    
    print(" & ".join(ans))

BiLSTM-UNK
['วัน', 'นี้', 'ฉัน', 'รู้สึก', 'เมือย', 'มาก']
NN & DDEM & PPER & VV & FXAV & JJA
['วัน', 'นี้', 'ฉัน', 'รูสึก', 'เมือย', 'มาก']
NN & DDEM & PPER & VV & FXAV & JJA
['วั', 'นี', 'ฉั', 'รู้สึก', 'เมือย', 'มา']
NN & NR & PU & VV & VV & VV
['วั', 'นี', 'ฉั', 'รูสึก', 'เมือย', 'มา']
NN & NR & PU & NR & VV & VV


In [40]:
print("ELMO")
for case in cases: 
    w_in = batch_to_ids([text]).to(device)
    pred_pos, pred_ner = model_elmo_pretrained_small_seed1(w_in)
    ans = []
    for pos in pred_pos:
        pos=pos.data.tolist()
        pos = pos.index(max(pos))
        pos = prep_torch.index_to_pos[pos]
        ans.append(pos)
    print(case)    
    print(" & ".join(ans))

ELMO
['วัน', 'นี้', 'ฉัน', 'รู้สึก', 'เมือย', 'มาก']
NN & DDEM & PPER & VV & VV & ADV
['วัน', 'นี้', 'ฉัน', 'รูสึก', 'เมือย', 'มาก']
NN & DDEM & PPER & VV & ADV & ADV
['วั', 'นี', 'ฉั', 'รู้สึก', 'เมือย', 'มา']
NN & DDEM & PPER & VV & ADV & ADV
['วั', 'นี', 'ฉั', 'รูสึก', 'เมือย', 'มา']
NN & DDEM & PPER & VV & VV & ADV


In [35]:
print("BCAD-clp")
for case in cases: 
    wtest_in = prep_torch.prepare_sequence_test_word(text)
    ctest_in = [prep_torch.prepare_sequence_test_char(word) for word in text]
    ptest_in = prep_torch.prepare_sequence_feat_prefix(text)  
    pred_pos, pred_ner = model_bilstm_BCAD_clp_seed1(wtest_in,ctest_in,ptest_in)
    ans = []
    for pos in pred_pos:
        pos=pos.data.tolist()
        pos = pos.index(max(pos))
        pos = prep_torch.index_to_pos[pos]
        ans.append(pos)
    print(case)    
    print(ans)    


BCAD-clp
['วัน', 'นี้', 'ฉัน', 'รู้สึก', 'เมือย', 'มาก']
['NN', 'DDEM', 'PPER', 'VV', 'VV', 'ADV']
['วัน', 'นี้', 'ฉัน', 'รูสึก', 'เมือย', 'มาก']
['NN', 'DDEM', 'PPER', 'VV', 'VV', 'ADV']
['วั', 'นี', 'ฉั', 'รู้สึก', 'เมือย', 'มา']
['NN', 'DDEM', 'PPER', 'VV', 'VV', 'ADV']
['วั', 'นี', 'ฉั', 'รูสึก', 'เมือย', 'มา']
['NN', 'DDEM', 'PPER', 'VV', 'VV', 'ADV']


In [36]:
print("UBC")
for case in cases: 
    wtest_in = prep_torch.prepare_sequence_test_word(text)
    ctest_in = [prep_torch.prepare_sequence_test_char(word) for word in text]
    ptest_in = prep_torch.prepare_sequence_feat_prefix(text)  
    pred_pos, pred_ner = model_bilstm_UBC_seed1(wtest_in,ctest_in,ptest_in)
    ans = []
    for pos in pred_pos:
        pos=pos.data.tolist()
        pos = pos.index(max(pos))
        pos = prep_torch.index_to_pos[pos]
        ans.append(pos)
    print(case)    
    print(ans)    


UBC
['วัน', 'นี้', 'ฉัน', 'รู้สึก', 'เมือย', 'มาก']
['NN', 'DDEM', 'PPER', 'VV', 'VV', 'ADV']
['วัน', 'นี้', 'ฉัน', 'รูสึก', 'เมือย', 'มาก']
['NN', 'DDEM', 'PPER', 'VV', 'VV', 'ADV']
['วั', 'นี', 'ฉั', 'รู้สึก', 'เมือย', 'มา']
['NN', 'DDEM', 'PPER', 'VV', 'VV', 'ADV']
['วั', 'นี', 'ฉั', 'รูสึก', 'เมือย', 'มา']
['NN', 'DDEM', 'PPER', 'VV', 'VV', 'ADV']


In [37]:
for case in cases:
    print(" & ".join(case))

วัน & นี้ & ฉัน & รู้สึก & เมือย & มาก
วัน & นี้ & ฉัน & รูสึก & เมือย & มาก
วั & นี & ฉั & รู้สึก & เมือย & มา
วั & นี & ฉั & รูสึก & เมือย & มา
